In [1]:
# Import necessary libraries
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# --- IMPORTANT: Your Database Credentials ---
db_user = 'root'
db_password = 'harsh'
db_host = 'localhost'
db_name = 'pune_real_estate_db'
table_name = 'raw_house_data'

In [3]:
# --- Create the Database Connection String ---
# This string tells pandas how to connect to your database.
db_connection_str = f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}'

In [4]:
try:
    # --- Create the Database Engine ---
    db_connection = create_engine(db_connection_str)

    # --- Load Data into a Pandas DataFrame ---
    # This query selects all data from your table.
    df = pd.read_sql(f'SELECT * FROM {table_name}', con=db_connection)

    # --- Verify the Import ---
    print("Successfully loaded data from SQL into DataFrame.")
    print(f"Shape of the DataFrame: {df.shape}") # (rows, columns)
    print("\nFirst 5 rows of the data:")
    print(df.head())

except Exception as e:
    print(f"An error occurred: {e}")
    print("\nPlease check the following:")
    print("1. Is your MySQL server running?")
    print("2. Did you update 'your_password' in the script correctly?")
    print("3. Is the database name and table name correct?")


Successfully loaded data from SQL into DataFrame.
Shape of the DataFrame: (12711, 10)

First 5 rows of the data:
   id             area_type   availability                  location  \
0   1  Super built-up  Area         19-Dec  Electronic City Phase II   
1   2            Plot  Area  Ready To Move          Chikka Tirupathi   
2   3        Built-up  Area  Ready To Move               Uttarahalli   
3   4  Super built-up  Area  Ready To Move        Lingadheeranahalli   
4   5  Super built-up  Area  Ready To Move                  Kothanur   

        size  society total_sqft  bath  balcony   price  
0      2 BHK  Coomee        1056   2.0      1.0   39.07  
1  4 Bedroom  Theanmp       2600   5.0      3.0  120.00  
2      3 BHK                1440   2.0      3.0   62.00  
3      3 BHK  Soiewre       1521   3.0      1.0   95.00  
4      2 BHK                1200   2.0      1.0   51.00  


In [5]:
# Make a copy to keep the original raw dataframe safe
df_clean = df.copy()

In [6]:
# --- 1. Handle Missing Values ---
# For 'bath' and 'balcony', we fill missing values with the median.
# The syntax below is the modern way to do this and avoids the FutureWarning.
df_clean['bath'] = df_clean['bath'].fillna(df_clean['bath'].median())
df_clean['balcony'] = df_clean['balcony'].fillna(df_clean['balcony'].median())

# Drop rows where 'location' or 'size' are missing, as they are critical.
df_clean.dropna(subset=['location', 'size'], inplace=True)

In [7]:
# --- 2. Clean the 'size' column and create 'bhk' ---
# The .str.split() method splits the string by space, and .str[0] gets the first part (the number).
# pd.to_numeric converts it to a number. 'coerce' turns any errors into NaN (Not a Number).
df_clean['bhk'] = pd.to_numeric(df_clean['size'].str.split().str[0], errors='coerce')
df_clean.dropna(subset=['bhk'], inplace=True) # Drop rows where conversion failed
df_clean['bhk'] = df_clean['bhk'].astype(int) # Convert to integer

In [8]:
# --- 3. Clean the 'total_sqft' column ---
# This function handles ranges like '1133 - 1384' by taking their average.
def convert_sqft_to_num(x):
    tokens = x.split('-')
    if len(tokens) == 2:
        try:
            # For a range, return the average
            return (float(tokens[0]) + float(tokens[1])) / 2
        except ValueError:
            return None # Return None if conversion fails
    try:
        # For a single number, just convert it
        return float(x)
    except (ValueError, TypeError):
        return None # Return None for other weird values

df_clean['total_sqft'] = df_clean['total_sqft'].apply(convert_sqft_to_num)
df_clean.dropna(subset=['total_sqft'], inplace=True) # Drop rows where conversion failed

In [9]:
# --- 4. Drop columns not needed for the model ---
df_model_ready = df_clean.drop(columns=['id', 'area_type', 'availability', 'size', 'society'])

In [10]:
# --- 5. Verify the cleaned data ---
print("Data cleaning and feature engineering complete.")
print(f"Shape of the final DataFrame: {df_model_ready.shape}")
print("\nFirst 5 rows of the cleaned data:")
print(df_model_ready.head())
print("\nData types of the columns:")
print(df_model_ready.info())

Data cleaning and feature engineering complete.
Shape of the final DataFrame: (12669, 6)

First 5 rows of the cleaned data:
                   location  total_sqft  bath  balcony   price  bhk
0  Electronic City Phase II      1056.0   2.0      1.0   39.07    2
1          Chikka Tirupathi      2600.0   5.0      3.0  120.00    4
2               Uttarahalli      1440.0   2.0      3.0   62.00    3
3        Lingadheeranahalli      1521.0   3.0      1.0   95.00    3
4                  Kothanur      1200.0   2.0      1.0   51.00    2

Data types of the columns:
<class 'pandas.core.frame.DataFrame'>
Index: 12669 entries, 0 to 12710
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   location    12669 non-null  object 
 1   total_sqft  12669 non-null  float64
 2   bath        12669 non-null  float64
 3   balcony     12669 non-null  float64
 4   price       12669 non-null  float64
 5   bhk         12669 non-null  int64  
dtypes: f

In [11]:
# Start with the dataframe from the previous step
df_prep = df_model_ready.copy()

In [12]:
# --- 1. Outlier Removal based on Business Logic ---
# It's unusual to have a house where the sq. ft. per bedroom is less than 300.
# We will remove such outliers.
df_prep = df_prep[~(df_prep['total_sqft'] / df_prep['bhk'] < 300)]

# It's also unusual for the number of bathrooms to be more than the number of bedrooms + 2.
df_prep = df_prep[df_prep.bath < df_prep.bhk + 2]

In [13]:
# --- 2. Feature Engineering: Create price_per_sqft ---
# This is a very important feature for identifying location-based price outliers.
# Price is in Lakhs, so we multiply by 100,000.
df_prep['price_per_sqft'] = df_prep['price'] * 100000 / df_prep['total_sqft']

In [14]:
# --- 3. Outlier Removal based on price_per_sqft ---
# For each location, we will remove data points that are beyond one standard deviation
# from the mean price_per_sqft for that location.
def remove_pps_outliers(df):
    df_out = pd.DataFrame()
    for key, subdf in df.groupby('location'):
        m = subdf.price_per_sqft.mean()
        st = subdf.price_per_sqft.std()
        reduced_df = subdf[(subdf.price_per_sqft > (m - st)) & (subdf.price_per_sqft < (m + st))]
        df_out = pd.concat([df_out, reduced_df], ignore_index=True)
    return df_out

df_prep = remove_pps_outliers(df_prep)

In [15]:
# --- 4. Prepare for Modeling: One-Hot Encoding for 'location' ---
# Machine learning models need all data to be numeric. We convert the 'location'
# column into many new columns, each representing a location.
dummies = pd.get_dummies(df_prep.location)

# To avoid the "dummy variable trap," we drop one column.
df_final = pd.concat([df_prep.drop('location', axis='columns'), dummies.drop('other', axis='columns', errors='ignore')], axis='columns')

In [16]:
# --- 5. Final Cleanup ---
# We can now drop the 'price_per_sqft' column as it was only used for outlier detection.
df_final = df_final.drop('price_per_sqft', axis='columns')

In [17]:
# --- 6. Verify the Final DataFrame ---
print("Outlier removal and final preparation complete.")
print(f"Shape of the final modeling DataFrame: {df_final.shape}")
print("\nFirst 5 rows of the final data:")
print(df_final.head())

Outlier removal and final preparation complete.
Shape of the final modeling DataFrame: (9151, 760)

First 5 rows of the final data:
   total_sqft  bath  balcony  price  bhk   Devarabeesana Halli  \
0      1100.0   2.0      1.0   70.0    2                  True   
1      1672.0   3.0      2.0  150.0    3                  True   
2      1750.0   3.0      3.0  149.0    3                  True   
3      1750.0   3.0      2.0  150.0    3                  True   
4      1250.0   2.0      3.0   44.0    3                 False   

    Devarachikkanahalli   Electronic City   Mysore Highway   Rachenahalli  \
0                 False             False            False          False   
1                 False             False            False          False   
2                 False             False            False          False   
3                 False             False            False          False   
4                  True             False            False          False   

   ...  

In [18]:
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import joblib
import json

In [19]:
# --- 1. Define Features (X) and Target (y) ---
X = df_final.drop('price', axis='columns')
y = df_final['price']

In [20]:
# --- 2. Split Data into Training and Testing Sets ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# --- 3. Initialize and Train the XGBoost Model ---
print("Training the XGBoost model...")
model = xgboost.XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.05, max_depth=5, subsample=0.8)
model.fit(X_train, y_train)
print("Model training complete.")

Training the XGBoost model...
Model training complete.


In [22]:
# --- 4. Evaluate the Model's Performance ---
y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
print("\n--- Model Evaluation ---")
print(f"R-squared (R2 Score): {r2:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f} Lakhs")
print("------------------------")


--- Model Evaluation ---
R-squared (R2 Score): 0.73
Mean Absolute Error (MAE): 18.03 Lakhs
------------------------


In [23]:
# --- 5. Save Model Files ---
joblib.dump(model, '../models/pune_house_price_model.pkl')
columns_data = {'columns': [col.lower() for col in X.columns.tolist()]}
with open('../models/model_columns.json', 'w') as f:
    json.dump(columns_data, f)
print("\nModel files saved successfully.")


Model files saved successfully.


In [24]:
# --- 6. (NEW) Save the Correct Data for the Dashboard ---
# We use the 'df_prep' DataFrame from the previous cell, before one-hot encoding.
dashboard_df = df_prep.drop(columns=['price_per_sqft'])
dashboard_csv_path = '../data/dashboard_data.csv'
dashboard_df.to_csv(dashboard_csv_path, index=False)
print(f"Dashboard data successfully saved to: {dashboard_csv_path}")

PermissionError: [Errno 13] Permission denied: '../data/dashboard_data.csv'